# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview

* This dataset is Heart Failure Prediction dataset taken from kaggle.

* Given the different features we need to predict if the patient will have heart failure.

* This is classification problem.

* [Link](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) to the dataset.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart-failure-prediction'
project_folder = './automl-hf-prediction'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
heart-failure-prediction,mltest,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "cluster-automl"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4,
                                                           min_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5) #timeout_in_minutes = 10

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure-dataset"
description_text = "Heart Failure Prediction data from kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/chinmaydas96/nd00333-capstone/master/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [5]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.25, random_state=1)

In [7]:
if "train_data_folder" not in os.listdir():
    os.mkdir("./train_data_folder")

if "models" not in os.listdir():
    os.mkdir("./models")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

train_df.to_csv("./train_data_folder/train.csv", index = False)

In [8]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./train_data_folder", target_path = 'datasets', overwrite = True,  show_progress = True)

Uploading an estimated of 1 files
Uploading ./train_data_folder/train.csv
Uploaded ./train_data_folder/train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_d1a90b1564d44d139aab37c6565625ca

In [9]:
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("datasets/train.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "n_cross_validations": 5,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",    
                             path = project_folder,
                             enable_onnx_compatible_models=True,
                             enable_early_stopping= True,
                             featurization= 'auto', #off
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

Submitting remote run.
No run_configuration provided, running on cluster-automl with default configuration
Running on remote compute: cluster-automl


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-prediction,AutoML_b3434009-d6ca-4b09-bb8f-6352e219dfb4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-prediction,AutoML_b3434009-d6ca-4b09-bb8f-6352e219dfb4,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_b3434009-d6ca-4b09-bb8f-6352e219dfb4',
 'target': 'cluster-automl',
 'status': 'Completed',
 'startTimeUtc': '2021-04-14T01:12:33.284142Z',
 'endTimeUtc': '2021-04-14T01:30:32.281839Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cluster-automl',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-prediction","subscription_id":"99625554-65d2-4421-aea7-7ddd513d8fe5","resource_group":"mltest","workspace_name":"mltest","region":"eastus2","compute_target":"cluster-automl","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.0,"n_cross_v

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_automlclass , best_model_automl = remote_run.get_output()
print(best_model_automl)
print(best_automlclass)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                max_iter=100,
                                                                                                multi_class='ovr',
                                                                                                n_jobs=1,
                                                             

In [14]:
best_automlclass.download_file('outputs/model.pkl', 'outputs/model.pkl')

In [ ]:
best_automlclass.download_file('outputs/scoring_file_v_1_0_0.py', 'score_aml.py')

In [16]:
model = best_automlclass.register_model(model_name = 'best_model_automl', model_path = './outputs/model.pkl')

In [17]:
best_automlclass.get_metrics()

{'weighted_accuracy': 0.8424486549964036,
 'precision_score_macro': 0.8442115632189162,
 'f1_score_weighted': 0.8368185742788142,
 'average_precision_score_weighted': 0.930488248537657,
 'recall_score_micro': 0.8381420765027322,
 'norm_macro_recall': 0.666329079669479,
 'balanced_accuracy': 0.8331645398347394,
 'average_precision_score_micro': 0.9212735870621731,
 'recall_score_macro': 0.8331645398347394,
 'accuracy': 0.8381420765027322,
 'f1_score_macro': 0.8331981762672676,
 'AUC_weighted': 0.9302478776336628,
 'precision_score_weighted': 0.8457822411146303,
 'AUC_micro': 0.924200901788647,
 'precision_score_micro': 0.8381420765027322,
 'AUC_macro': 0.9302478776336628,
 'f1_score_micro': 0.8381420765027322,
 'average_precision_score_macro': 0.9285846720633604,
 'log_loss': 0.46170742159165173,
 'matthews_correlation': 0.676964879074445,
 'recall_score_weighted': 0.8381420765027322,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_b3434009-d6ca-4b09-bb8f-6352e219dfb4_3

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
from azureml.core.environment import Environment
environment = Environment.get(ws, "AzureML-AutoML")

In [19]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model

inference_config = InferenceConfig(entry_script='score_aml.py',
                                   environment=environment)
service_name = 'automl-deploy-1'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-14 01:34:04+00:00 Creating Container Registry if not exists.
2021-04-14 01:34:05+00:00 Registering the environment.
2021-04-14 01:34:08+00:00 Use the existing image.
2021-04-14 01:34:09+00:00 Submitting deployment to compute..
2021-04-14 01:34:13+00:00 Checking the status of deployment automl-deploy-1..
2021-04-14 01:37:15+00:00 Checking the status of inference endpoint automl-deploy-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://1e2639dc-d74e-4d88-9ec6-b9f66afed09a.eastus2.azurecontainer.io/score


In [20]:
# Enable app insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
import requests
import json

# URL for the web service, should be similar to:
scoring_uri = 'http://1e2639dc-d74e-4d88-9ec6-b9f66afed09a.eastus2.azurecontainer.io/score'
# If the service is authenticated, set the key or token
#key = 'zuv0yY9prOPHFssuYSFWDJPhnVMxgJqG'
#two set of data to score, so we get two results back
data = {"data":
        [
          {
            "age":37,
            "sex": 1,
            "cp": 2,
            "trestbps": 130,
            "chol": 250,
            "fbs": 0,
            "restecg": 1,
            "thalach": 187,
            "exang": 0,
            "oldpeak": 3.5,
            "slope": 0,
            "ca": 0,
            "thal": 2,
          }
      ]
    }
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


{"result": [1]}


In [23]:
 data = [
          {
            "age":37,
            "sex": 1,
            "cp": 2,
            "trestbps": 130,
            "chol": 250,
            "fbs": 0,
            "restecg": 1,
            "thalach": 187,
            "exang": 0,
            "oldpeak": 3.5,
            "slope": 0,
            "ca": 0,
            "thal": 2,
          }
      ]
    

In [24]:
# test using service instance
input_data = json.dumps({
    'data': data
})

output = service.run(input_data)
output

'{"result": [1]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
logs = service.get_logs()
logs

'2021-04-14T01:37:09,894826400+00:00 - gunicorn/run \n2021-04-14T01:37:09,909107300+00:00 - iot-server/run \n2021-04-14T01:37:09,902095300+00:00 - rsyslog/run \n2021-04-14T01:37:09,918677000+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [ ]:
service.delete()